In [1]:
#hide

%pip install pandas
%pip install numpy
%pip install scikit-learn==1.5.2
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 113.8 MB/s eta 0:00:0000:01:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.0
    Uninstalling scikit-learn-1.3.0:
      Successfully uninstalled scikit-learn-1.3.0
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

In [3]:
df_regression = pd.read_csv("./train_taxi.csv")

In [4]:
df_regression.head()

,ID,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,...,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,trip_type,congestion_surcharge,payment_type
0,T03315,2.0,2021-07-02 17:19:11,2021-07-02 17:40:02,N,1.0,152,142,1.0,4.30,...,1.0,0.5,5.00,0.00,NaN,0.3,27.55,1.0,2.75,1.0
1,T07720,2.0,2021-07-05 22:48:04,2021-07-05 23:12:02,N,1.0,93,107,2.0,13.52,...,0.5,0.5,12.15,6.55,NaN,0.3,60.75,1.0,2.75,1.0
2,T09695,2.0,2021-07-07 06:29:43,2021-07-07 06:38:42,N,1.0,74,75,1.0,1.55,...,0.0,0.5,0.00,0.00,NaN,0.3,8.80,1.0,0.00,2.0
3,T08802,2.0,2021-07-06 15:33:29,2021-07-06 15:43:25,N,1.0,226,129,1.0,1.22,...,0.0,0.5,0.00,0.00,NaN,0.3,8.80,1.0,0.00,2.0
4,T01413,2.0,2021-07-01 17:09:07,2021-07-01 18:03:12,N,1.0,55,86,1.0,14.23,...,1.0,0.5,0.00,2.45,NaN,0.3,49.75,1.0,0.00,1.0


In [5]:
df_regression.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     8000 non-null   object 
 1   VendorID               8000 non-null   float64
 2   lpep_pickup_datetime   8000 non-null   object 
 3   lpep_dropoff_datetime  8000 non-null   object 
 4   store_and_fwd_flag     8000 non-null   object 
 5   RatecodeID             8000 non-null   float64
 6   PULocationID           8000 non-null   int64  
 7   DOLocationID           8000 non-null   int64  
 8   passenger_count        8000 non-null   float64
 9   trip_distance          8000 non-null   float64
 10  fare_amount            8000 non-null   float64
 11  extra                  8000 non-null   float64
 12  mta_tax                8000 non-null   float64
 13  tip_amount             8000 non-null   float64
 14  tolls_amount           8000 non-null   float64
 15  ehai

In [6]:
df_regression = df_regression.drop(columns=['ehail_fee'], axis=1)

In [7]:
df_regression.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     8000 non-null   object 
 1   VendorID               8000 non-null   float64
 2   lpep_pickup_datetime   8000 non-null   object 
 3   lpep_dropoff_datetime  8000 non-null   object 
 4   store_and_fwd_flag     8000 non-null   object 
 5   RatecodeID             8000 non-null   float64
 6   PULocationID           8000 non-null   int64  
 7   DOLocationID           8000 non-null   int64  
 8   passenger_count        8000 non-null   float64
 9   trip_distance          8000 non-null   float64
 10  fare_amount            8000 non-null   float64
 11  extra                  8000 non-null   float64
 12  mta_tax                8000 non-null   float64
 13  tip_amount             8000 non-null   float64
 14  tolls_amount           8000 non-null   float64
 15  impr

In [8]:
df_regression = df_regression.drop(columns=['ID'])

In [9]:
df_regression.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_type,congestion_surcharge,payment_type
0,2.0,2021-07-02 17:19:11,2021-07-02 17:40:02,N,1.0,152,142,1.0,4.30,18.0,1.0,0.5,5.00,0.00,0.3,27.55,1.0,2.75,1.0
1,2.0,2021-07-05 22:48:04,2021-07-05 23:12:02,N,1.0,93,107,2.0,13.52,38.0,0.5,0.5,12.15,6.55,0.3,60.75,1.0,2.75,1.0
2,2.0,2021-07-07 06:29:43,2021-07-07 06:38:42,N,1.0,74,75,1.0,1.55,8.0,0.0,0.5,0.00,0.00,0.3,8.80,1.0,0.00,2.0
3,2.0,2021-07-06 15:33:29,2021-07-06 15:43:25,N,1.0,226,129,1.0,1.22,8.0,0.0,0.5,0.00,0.00,0.3,8.80,1.0,0.00,2.0
4,2.0,2021-07-01 17:09:07,2021-07-01 18:03:12,N,1.0,55,86,1.0,14.23,45.5,1.0,0.5,0.00,2.45,0.3,49.75,1.0,0.00,1.0


In [10]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

df_regression["store_and_fwd_flag"] = label_encoder.fit_transform(df_regression["store_and_fwd_flag"])



In [11]:
df_regression['pickup_datetime'] = pd.to_datetime(df_regression['lpep_pickup_datetime']) 
df_regression['dropoff_datetime'] = pd.to_datetime(df_regression['lpep_dropoff_datetime'])

In [12]:
df_regression['pickup_hour'] = df_regression['pickup_datetime'].dt.hour
df_regression['pickup_day'] = df_regression['pickup_datetime'].dt.dayofweek
df_regression['pickup_month'] = df_regression['pickup_datetime'].dt.month
df_regression['dropoff_hour'] = df_regression['dropoff_datetime'].dt.hour
df_regression['dropoff_day'] = df_regression['dropoff_datetime'].dt.dayofweek
df_regression['dropoff_month'] = df_regression['dropoff_datetime'].dt.month

In [13]:
df_regression['trip_duration'] = (df_regression['dropoff_datetime'] - df_regression['pickup_datetime']).dt.total_seconds() / 60

In [14]:
def encode_cyclical(feature, max_val):
    df_regression[f'{feature}_sin'] = np.sin(2 * np.pi * df_regression[feature] / max_val)
    df_regression[f'{feature}_cos'] = np.cos(2 * np.pi * df_regression[feature] / max_val)

In [15]:
encode_cyclical('pickup_hour', 24)
encode_cyclical('dropoff_hour', 24)
encode_cyclical('pickup_month', 12)
encode_cyclical('dropoff_month', 12)

In [16]:
df_regression = df_regression.drop(columns=['pickup_hour', 'pickup_month', 'dropoff_hour', 'dropoff_month', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'pickup_datetime', 'dropoff_datetime'], axis=1)

In [17]:
df_regression.head()

,VendorID,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,...,dropoff_day,trip_duration,pickup_hour_sin,pickup_hour_cos,dropoff_hour_sin,dropoff_hour_cos,pickup_month_sin,pickup_month_cos,dropoff_month_sin,dropoff_month_cos
0,2.0,0,1.0,152,142,1.0,4.30,18.0,1.0,0.5,...,4,20.850000,-0.965926,-2.588190e-01,-0.965926,-2.588190e-01,-0.5,-0.866025,-0.5,-0.866025
1,2.0,0,1.0,93,107,2.0,13.52,38.0,0.5,0.5,...,0,23.966667,-0.500000,8.660254e-01,-0.258819,9.659258e-01,-0.5,-0.866025,-0.5,-0.866025
2,2.0,0,1.0,74,75,1.0,1.55,8.0,0.0,0.5,...,2,8.983333,1.000000,6.123234e-17,1.000000,6.123234e-17,-0.5,-0.866025,-0.5,-0.866025
3,2.0,0,1.0,226,129,1.0,1.22,8.0,0.0,0.5,...,1,9.933333,-0.707107,-7.071068e-01,-0.707107,-7.071068e-01,-0.5,-0.866025,-0.5,-0.866025
4,2.0,0,1.0,55,86,1.0,14.23,45.5,1.0,0.5,...,3,54.083333,-0.965926,-2.588190e-01,-1.000000,-1.836970e-16,-0.5,-0.866025,-0.5,-0.866025


In [18]:
pickup_day_one_hot = pd.get_dummies(df_regression['pickup_day'], prefix='pickup_day', drop_first=True).astype(int)
dropoff_day_one_hot = pd.get_dummies(df_regression['dropoff_day'], prefix='dropoff_day', drop_first=True).astype(int)

In [19]:
df_regression = pd.concat([df_regression, pickup_day_one_hot, dropoff_day_one_hot], axis=1) 

In [20]:
df_regression = df_regression.drop(columns=['pickup_day', 'dropoff_day'], axis=1)


In [21]:
df_regression.head()

,VendorID,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,...,pickup_day_3,pickup_day_4,pickup_day_5,pickup_day_6,dropoff_day_1,dropoff_day_2,dropoff_day_3,dropoff_day_4,dropoff_day_5,dropoff_day_6
0,2.0,0,1.0,152,142,1.0,4.30,18.0,1.0,0.5,...,0,1,0,0,0,0,0,1,0,0
1,2.0,0,1.0,93,107,2.0,13.52,38.0,0.5,0.5,...,0,0,0,0,0,0,0,0,0,0
2,2.0,0,1.0,74,75,1.0,1.55,8.0,0.0,0.5,...,0,0,0,0,0,1,0,0,0,0
3,2.0,0,1.0,226,129,1.0,1.22,8.0,0.0,0.5,...,0,0,0,0,1,0,0,0,0,0
4,2.0,0,1.0,55,86,1.0,14.23,45.5,1.0,0.5,...,1,0,0,0,0,0,1,0,0,0


In [22]:
import xgboost as xgb # type: ignore
from sklearn.model_selection import RandomizedSearchCV, train_test_split #type: ignore
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score #type: ignore


In [23]:
X = df_regression.drop(columns='trip_distance', axis=1)
y = df_regression['trip_distance']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 300, 500, 1000],
    'max_depth': [3, 5, 7, 9, 11],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3, 0.4],
    'reg_alpha': [0, 0.1, 0.2, 0.3, 0.4],
    'reg_lambda': [0.1, 0.5, 1.0, 2.0, 3.0]
}

model = xgb.XGBRegressor(objective='reg:squarederror')
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                                   n_iter=100, scoring='neg_root_mean_squared_error',
                                   cv=5, verbose=1, n_jobs=-1, random_state=42)


In [26]:
random_search.fit(X_train, y_train)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


KeyboardInterrupt: 

In [196]:
best_params = random_search.best_params_
print(best_params)

best_model = random_search.best_estimator_
print(best_model)

{'subsample': 0.6, 'reg_lambda': 0.1, 'reg_alpha': 0.2, 'n_estimators': 1000, 'max_depth': 7, 'learning_rate': 0.05, 'gamma': 0.3, 'colsample_bytree': 0.8}
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.3, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)


In [173]:
y_pred = best_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae:.2f}')
print(f'Mean Squared Error: {mse:.2f}')
print(f'Root Mean Squared Error: {rmse:.2f}')
print(f'R-squared: {r2:.2f}')

Mean Absolute Error: 0.34
Mean Squared Error: 0.63
Root Mean Squared Error: 0.80
R-squared: 0.97


In [174]:
df_train_regression = pd.read_csv("./taxi_trip_regress_new.csv")


In [175]:
df_train_regression.head()

,Unnamed: 0,ID,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,...,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,trip_type,congestion_surcharge,payment_type
0,0,T00876,1.0,2021-07-01 13:41:08,2021-07-01 14:36:53,N,2.0,74,132,1.0,...,0.0,0.5,10.00,6.55,NaN,0.3,69.35,1.0,0.00,1.0
1,1,T01498,2.0,2021-07-01 18:33:34,2021-07-01 19:20:39,N,5.0,82,143,1.0,...,0.0,0.0,2.46,6.55,NaN,0.3,34.08,1.0,2.75,1.0
2,2,T08153,2.0,2021-07-06 09:37:59,2021-07-06 09:46:37,N,1.0,74,75,1.0,...,0.0,0.5,2.08,0.00,NaN,0.3,10.38,1.0,0.00,1.0
3,3,T03394,2.0,2021-07-02 18:24:34,2021-07-02 18:39:57,N,1.0,244,50,1.0,...,1.0,0.5,4.91,0.00,NaN,0.3,29.46,1.0,2.75,1.0
4,4,T09784,2.0,2021-07-07 08:30:59,2021-07-07 08:40:24,N,1.0,7,129,1.0,...,0.0,0.5,0.00,0.00,NaN,0.3,10.80,1.0,0.00,2.0


In [176]:
df_train_regression = df_train_regression.drop(columns=['Unnamed: 0', 'ehail_fee'])

In [177]:
df_train_regression.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     400 non-null    object 
 1   VendorID               400 non-null    float64
 2   lpep_pickup_datetime   400 non-null    object 
 3   lpep_dropoff_datetime  400 non-null    object 
 4   store_and_fwd_flag     400 non-null    object 
 5   RatecodeID             400 non-null    float64
 6   PULocationID           400 non-null    int64  
 7   DOLocationID           400 non-null    int64  
 8   passenger_count        400 non-null    float64
 9   fare_amount            400 non-null    float64
 10  extra                  400 non-null    float64
 11  mta_tax                400 non-null    float64
 12  tip_amount             400 non-null    float64
 13  tolls_amount           400 non-null    float64
 14  improvement_surcharge  400 non-null    float64
 15  total_

In [178]:
processed_df_train_regression = df_train_regression.copy()

In [179]:
processed_df_train_regression["store_and_fwd_flag"] = label_encoder.transform(processed_df_train_regression["store_and_fwd_flag"])

In [180]:
processed_df_train_regression

,ID,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_type,congestion_surcharge,payment_type
0,T00876,1.0,2021-07-01 13:41:08,2021-07-01 14:36:53,0,2.0,74,132,1.0,52.00,0.0,0.5,10.00,6.55,0.3,69.35,1.0,0.00,1.0
1,T01498,2.0,2021-07-01 18:33:34,2021-07-01 19:20:39,0,5.0,82,143,1.0,22.02,0.0,0.0,2.46,6.55,0.3,34.08,1.0,2.75,1.0
2,T08153,2.0,2021-07-06 09:37:59,2021-07-06 09:46:37,0,1.0,74,75,1.0,7.50,0.0,0.5,2.08,0.00,0.3,10.38,1.0,0.00,1.0
3,T03394,2.0,2021-07-02 18:24:34,2021-07-02 18:39:57,0,1.0,244,50,1.0,20.00,1.0,0.5,4.91,0.00,0.3,29.46,1.0,2.75,1.0
4,T09784,2.0,2021-07-07 08:30:59,2021-07-07 08:40:24,0,1.0,7,129,1.0,10.00,0.0,0.5,0.00,0.00,0.3,10.80,1.0,0.00,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,T07978,2.0,2021-07-06 08:58:05,2021-07-06 09:24:58,0,1.0,75,136,1.0,25.00,0.0,0.5,3.00,0.00,0.3,28.80,1.0,0.00,1.0
396,T09337,2.0,2021-07-06 18:15:57,2021-07-06 18:16:50,0,1.0,193,193,1.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,1.0,0.00,1.0
397,T03284,1.0,2021-07-02 17:52:27,2021-07-02 17:59:55,0,1.0,95,95,1.0,6.50,1.0,0.5,1.70,0.00,0.3,10.00,1.0,0.00,1.0
398,T07319,2.0,2021-07-05 17:14:54,2021-07-05 17:25:24,0,1.0,95,95,1.0,9.50,0.0,0.5,0.00,0.00,0.3,10.30,1.0,0.00,2.0


In [181]:
processed_df_train_regression['pickup_datetime'] = pd.to_datetime(processed_df_train_regression['lpep_pickup_datetime'])
processed_df_train_regression['dropoff_datetime'] = pd.to_datetime(processed_df_train_regression['lpep_dropoff_datetime'])

In [182]:
processed_df_train_regression['pickup_hour'] = processed_df_train_regression['pickup_datetime'].dt.hour
processed_df_train_regression['pickup_day'] = processed_df_train_regression['pickup_datetime'].dt.dayofweek
processed_df_train_regression['pickup_month'] = processed_df_train_regression['pickup_datetime'].dt.month
processed_df_train_regression['dropoff_hour'] = processed_df_train_regression['dropoff_datetime'].dt.hour
processed_df_train_regression['dropoff_day'] = processed_df_train_regression['dropoff_datetime'].dt.dayofweek
processed_df_train_regression['dropoff_month'] = processed_df_train_regression['dropoff_datetime'].dt.month


In [183]:
processed_df_train_regression['trip_duration'] = (processed_df_train_regression['dropoff_datetime'] - processed_df_train_regression['pickup_datetime']).dt.total_seconds() / 60

In [184]:
def encode_test_cyclical(feature, max_val):
    processed_df_train_regression[f'{feature}_sin'] = np.sin(2 * np.pi * processed_df_train_regression[feature] / max_val)
    processed_df_train_regression[f'{feature}_cos'] = np.cos(2 * np.pi * processed_df_train_regression[feature] / max_val)

In [185]:
encode_test_cyclical('pickup_hour', 24)
encode_test_cyclical('dropoff_hour', 24)
encode_test_cyclical('pickup_month', 12)
encode_test_cyclical('dropoff_month', 12)

In [186]:
processed_df_train_regression = processed_df_train_regression.drop(columns=['pickup_hour', 'pickup_month', 'dropoff_hour', 'dropoff_month', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'pickup_datetime', 'dropoff_datetime'], axis=1)

In [187]:
test_pickup_day_one_hot = pd.get_dummies(processed_df_train_regression['pickup_day'], prefix='pickup_day', drop_first=True).astype(int)
test_dropoff_day_one_hot = pd.get_dummies(processed_df_train_regression['dropoff_day'], prefix='dropoff_day', drop_first=True).astype(int)

In [188]:
processed_df_train_regression = pd.concat([processed_df_train_regression, test_pickup_day_one_hot, test_dropoff_day_one_hot], axis=1)

In [189]:
processed_df_train_regression = processed_df_train_regression.drop(columns=['pickup_day', 'dropoff_day'], axis=1)

In [190]:
processed_df_train_regression.head()

,ID,VendorID,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,fare_amount,extra,mta_tax,...,pickup_day_3,pickup_day_4,pickup_day_5,pickup_day_6,dropoff_day_1,dropoff_day_2,dropoff_day_3,dropoff_day_4,dropoff_day_5,dropoff_day_6
0,T00876,1.0,0,2.0,74,132,1.0,52.00,0.0,0.5,...,1,0,0,0,0,0,1,0,0,0
1,T01498,2.0,0,5.0,82,143,1.0,22.02,0.0,0.0,...,1,0,0,0,0,0,1,0,0,0
2,T08153,2.0,0,1.0,74,75,1.0,7.50,0.0,0.5,...,0,0,0,0,1,0,0,0,0,0
3,T03394,2.0,0,1.0,244,50,1.0,20.00,1.0,0.5,...,0,1,0,0,0,0,0,1,0,0
4,T09784,2.0,0,1.0,7,129,1.0,10.00,0.0,0.5,...,0,0,0,0,0,1,0,0,0,0


In [191]:
processed_df_train_regression = processed_df_train_regression.drop(columns="ID")

In [192]:
y_result = best_model.predict(processed_df_train_regression)

In [193]:
csv_result = pd.DataFrame({
    "ID": df_train_regression["ID"],
    "trip_distance": y_result
})

In [194]:
csv_result

,ID,trip_distance
0,T00876,11.542068
1,T01498,4.312709
2,T08153,1.327783
3,T03394,6.136438
4,T09784,2.316581
...,...,...
395,T07978,7.043583
396,T09337,-0.095228
397,T03284,0.995123
398,T07319,2.028426


In [195]:
from datetime import datetime
from zoneinfo import ZoneInfo

utc_plus_7 = ZoneInfo("Asia/Bangkok")
current_iso_timestamp = datetime.now(utc_plus_7).isoformat()

filename = f"submission_kasdead_{current_iso_timestamp}.csv"
csv_result.to_csv(filename, index=False)